# Preprocessing

Folgende Schritte werden hier durchgeführt:

1. Laden der Daten
2. Songs filtern, welche nicht verwendet werden können
3. Transponieren der Lieder zu einheitlichem C maj oder A min
4. Encode die Musik zu einem Zeitserienformat
5. Speicher die Zeitserien in eine Textdatei für spätere wiederverwendung

## Laden der Songs
In dem Abschnitt werden alle Daten geladen die in dem Ornder `deutschl`sind mit der Endung `.krn`

In [1]:
from pathlib import Path
import music21 as m21

Laden der Lieder in `songs` in einem Music21 Score.

In [2]:
data_folder = Path(__file__).parent / 'deutschl' / 'test'

songs = []

# Open all .krn files in the folder
for file in data_folder.glob('*.krn'):
    song = m21.converter.parse(file)
    songs.append(song)

Anzahl der geladenen Lieder:

In [3]:
print(f"Loaded {len(songs)} songs")

Loaded 12 songs


Zeigt das Lied 0 an. </br>
Hinweis msuic21 muss entsprechend eingestellt sein

In [4]:
# songs[0].show()

## Songs filtern
Filtern der Songs nach akzeptable Längen der Noten:
- 0.25
- 0.5
- 0.75
- 1.0
- 1.5
- 2
- 3
- 4

Siehe hier (3 und 0.75 sind Noten mit nem Punkt hinten drann hier nicht gelisted):
![](./Bilder/Notenl%C3%A4nge.png)

In [5]:
accaptable_durations = [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0]

bad_songs = []

for index, song in enumerate(songs):
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in accaptable_durations:
            bad_songs.append(index)
            break

print(f"Found {len(bad_songs)} bad songs")

# Remove bad songs
for index in sorted(bad_songs, reverse=True):
    del songs[index]

print(f"Removed {len(bad_songs)} bad songs")
print(f"Remaining songs: {len(songs)}")

Found 0 bad songs
Removed 0 bad songs
Remaining songs: 12


## Transponieren der Lieder zu einheitlichem C maj oder A min

Dies wird gemacht, um später zum lernen der KI Daten einzusparen. Nach diesem Schritt muss das Netzwerk nicht alle 24 verschiedene Schlüssel generalliesiern. Möchte man es torztdem, dann nimmnt man alle Daten und trnsponiert es in alle verschiedene Notenschlüssel und verwendet dies zum lernen. Das sind aber ein haufen Daten, die man an sich nicht benötigt

1. Notenschlüssel des Liedes bekommen / ermittelen </br>
   Parts sind die einzelnen Abschnitte der Lieder
2. Entweder die Notenschlüssel ist bereits im Lied hinterlegt -> Verwende diesen </br>
   Oder music21 versucht den Notenschlüssel herauszufinden
3. Wir berechnen den Abstand von einem major Lied zu C oder von minor nach A
4. Transponieren alle Lieder mit dem zuvor ermittelten Notenschlüssel

----

Notizen

Maj -> Dur (glücklich)
Min -> Moll (unglücklich)

Music21 Aufbau
![](./Bilder/music21-Struktur.png)
![](./Bilder/music21-Struktur-Beispiel.png)

In [6]:
transposed_songs = []

for song in songs:
    parts = song.getElementsByClass(m21.stream.Part)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4]      # There is the key signature of the most songs in THIS dataset 

    # If no key siganture is found, we use music21's key guessing algorithm
    if not isinstance(key, m21.key.Key):
        key = song.analyze('key')

    # get interval for transposition
    if key.mode == "major":
        # Interval between tonic and C
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        # Interval between tonic and A
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))

    # transpose song to C major or A minor
    transposed_song = song.transpose(interval)
    transposed_songs.append(transposed_song)

In [7]:
# transposed_songs[0].show()

## Encoden der Songs
Die Songs werden in ein Textformat transponiert.

Hierfür verwendet man die MIDI Integer Werte für die einzelne Tonhöhen. Jeder Zeitspempel in der Textdatei ist eine 1/16 Note. Bedeutet, dass bei der Tonhöhe von 60 und einer Länge von 1.0 Takte dies umgeandelt wird in -> [60, "_", "_", "_"]

Ein "_" bedeutet, dass der Vorherige gespielte Ton immer noch gespielt wird.

In [8]:
encoded_songs = []

for song in transposed_songs:

    # Every time step is a quarter note
    time_step = 0.25

    encoded_song = []

    # Save the song in MIDI format
    # A event is a note or a rest
    for event in song.flat.notesAndRests:

        # Notes
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi
        # Rests
        elif isinstance(event, m21.note.Rest):
            symbol = "r"

        # For example, if the duration of the event is 1.0 (a quarter note), we need to add 4 time steps
        # The note itself and 3 "_" symbols
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")

    # cast the encoded song to string
    encoded_song = " ".join(map(str, encoded_song))
    encoded_songs.append(encoded_song)

So sehen die Songs jetzt aus:

In [9]:
print(encoded_songs[0])

64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ r _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ r _ _ _


## Speichern der Encoded Songs
Um die Schritte nicht immer wieder erneut durchzuführen, speichern wir die `encoded_songs` im Ordner datasets

In [10]:
# Create the folders
datasets_folder = Path(__file__).parent / 'dataset'
datasets_folder.mkdir(exist_ok=True)

print(datasets_folder)

D:\Dokumente2\Python_Programme\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\demos\dataset


In [11]:
# Clear the folder from old Data
for file in datasets_folder.glob('*'):
    file.unlink()

In [12]:
for index, encoded_song in enumerate(encoded_songs):
    save_path = datasets_folder / f"{index}"
    with open(save_path, "w") as file:
        file.write(encoded_song)

## Erstellen einer einzigen Datei der Trainingsdaten
Um das Netzwerk später leichter zu trainieren, speichern wir alle Songs in einer einzigen Datei. Die einzelne Songs werden mit einem Sepperator getrennt.

In [13]:
# Load the data
encoded_songs = []
for file in datasets_folder.glob('*'):
    with open(file, "r") as f:
        encoded_songs.append(f.read())

In [14]:
single_file = datasets_folder / 'singel_file'
single_file.unlink(missing_ok=True)

sequence_length = 64

new_song_delimiter = "/ " * sequence_length

So schaut ein song aus, wie er in die Datei gespeichert wird:

In [15]:
song_0 = encoded_songs[0] + " " + new_song_delimiter
print(f"Length: {len(song_0)}")
print(song_0)

Length: 351
64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ r _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ r _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 


In [16]:
# Create a single file with all songs
with open(single_file, "a") as file:
    for index, song in enumerate(encoded_songs):
        file.write(song)

        # Remove the last space if it is the last song
        if index != len(encoded_songs) - 1:
            file.write( " " + new_song_delimiter)
        else:
            file.write( " " + new_song_delimiter[:-1])

In [17]:
# Show the first 1000 characters
with open(single_file, "r") as file:
    print(file.read(1000))

64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ r _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ r _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ r _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ r _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 55 _ _ _ 64 _ _ _ 64 _ 64 _ 64 _ _ _ 62 _ 62 _ 62 _ _ _ _ _ 60 _ 60 _ _ _ 55 _ _ _ 65 _ _ _ 65 _ 65 _ 65 _ 64 _ 67 _ 64 _ 64 _ _ _ 62 _ _ _ r _ _ _ 55 _ _ _ 67 _ _ _ 67 _ 67 _ 67 _ _ _ 65 _ 65 _ 64 _ _ _ _ _ _ _ 67 _ 65 _ 64 _ 62 _ 60 _ _ _ 60 _ 60 _ 64 _ 62 _ 62 _ _ _ 60 _ _ _ _ _ _ _ 62 _ _ _ 65

In [18]:
# Show the last 1000 characters
with open(single_file, "r") as file:
    text = file.read()
    print(text[-1000:])

/ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 76 _ _ _ _ _ 74 _ 72 _ 64 _ 69 _ _ _ 67 _ 65 _ 62 _ _ _ 69 _ _ _ 67 _ 65 _ 62 _ _ _ 69 _ _ _ 67 _ 64 _ 60 _ _ r 76 _ _ _ _ _ 74 _ 72 _ 64 _ 69 _ _ _ 67 _ 65 _ 62 _ _ _ 67 _ _ _ 69 _ _ _ 71 _ _ _ 72 _ _ _ _ _ _ _ r _ _ _ 77 _ _ 76 74 _ _ _ r _ _ _ 76 _ _ 74 72 _ _ _ r _ _ _ 67 _ _ _ 69 _ _ _ 71 _ _ _ 72 _ 71 _ 72 _ 74 _ 76 _ _ _ 77 _ _ _ 76 74 _ _ _ r _ _ 76 _ _ 74 72 _ _ _ r _ _ _ 67 _ _ _ 69 _ _ _ 71 _ _ _ 72 _ _ _ _ _ _ _ r _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ r _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ r _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / /

## Erstellen einer Lookup Tabelle
Jedes Zeichen in der `single_file` muss gemapped werden in einen Zahlenwert. Nur so kann das die KI verarbeiten.

Dies wird realisiert, indem wir ein `Dict` erstellen.
Dabei wir die "Space" jedoch nicht mehr berücksichtigt!

In [19]:
mappings = {}

songs = ""
with open(single_file, "r") as file:
    songs = file.read()

# Create a set with all unique characters
songs_splittet = songs.split()
vocabular = sorted(set(songs_splittet))

In [20]:
print(f"Vocabular: {vocabular}")

Vocabular: ['/', '55', '57', '60', '62', '64', '65', '67', '68', '69', '71', '72', '74', '76', '77', '81', '_', 'r']


In [21]:
# Create a dictionary to map characters to integers
for index, char in enumerate(vocabular):
    mappings[char] = index

In [22]:
# Save the dictionary for later use
import json

mapping_path = datasets_folder / 'mappings.json'
with open(mapping_path, "w") as file:
    json.dump(mappings, file, indent=4)

## Song in int Werte Konvertieren
Mithilfe der Lookuptabelle werden die Daten in Intdaten konventiert

In [23]:
# Load the mappings
mappings = {}
with open(mapping_path, "r") as file:
    mappings = json.load(file)

In [24]:
print(mappings)

{'/': 0, '55': 1, '57': 2, '60': 3, '62': 4, '64': 5, '65': 6, '67': 7, '68': 8, '69': 9, '71': 10, '72': 11, '74': 12, '76': 13, '77': 14, '81': 15, '_': 16, 'r': 17}


In [25]:
songs_in_int = []

for song in songs_splittet:
    songs_in_int.append(mappings[song])

Alle Songs sind nun komplett in Integer-Werte Konventiert.

In [26]:
print(songs_in_int[:100])

[5, 16, 9, 16, 16, 16, 10, 16, 11, 16, 16, 10, 9, 16, 13, 16, 16, 16, 16, 16, 10, 16, 16, 16, 12, 16, 11, 16, 16, 10, 9, 16, 8, 16, 16, 16, 9, 16, 10, 16, 16, 16, 16, 16, 17, 16, 16, 16, 5, 16, 13, 16, 16, 16, 13, 16, 11, 16, 16, 10, 9, 16, 15, 16, 16, 16, 16, 16, 14, 16, 16, 16, 12, 16, 11, 16, 16, 10, 9, 16, 10, 16, 16, 9, 8, 16, 9, 16, 16, 16, 16, 16, 17, 16, 16, 16, 0, 0, 0, 0]


In [27]:
# Save the songs
import numpy as np

songs_in_int = np.array(songs_in_int)

np.save(datasets_folder / 'songs.npy', songs_in_int)